In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from desert import isnum
from pfunctions import getTimeSteps
from FGMTableV2 import *
from readOFFiles import *
from scipy.interpolate import interp1d
from tableProperties import FGMtableProperties
from lookup import lookup
from oneDCaseReader import oneDCaseReader
import pickle

In [ ]:
FGMtableDict = {}

caseRoute         = "../CASEdf/03.2D_DNS_mapFields/"

FGMtableDict["route"] = caseRoute
FGMtableDict["Zrange"] = (0.,1.)
FGMtableDict["fuelList"] = ["C7H16","NH3"]
FGMtableDict["gridNumber"] = (301,501)
FGMtableDict["gridPower"] = (2,1)
FGMtableDict["extraLookupFields"] = ["Qdot", "OH","C7H15O2"]
FGMtableDict["TimeStepsDict"] = {"lowerBound":0.,"upperBound":0.006,"stepEvery":1}
# FGMtableDict["PVFields"] = ["N2","CO2","CO","HO2","CH2O","H2O"]
# FGMtableDict["PVCoeffs"] = [1.,1.2, 0.9, 2.7, 1.5, 1.2]
# FGMtableDict["PVConstant"] = 1.
FGMtableDict["PVFields"] = ["CO2","CO","HO2","CH2O","H2O"]
FGMtableDict["PVCoeffs"] = [1.2, 0.9, 2.7, 1.5, 1.2]
FGMtableDict["PVConstant"] = 0.


obj = FGMtable(FGMtableDict)
obj.Allrun("./","./table_m2a5000/")

In [ ]:
obj.statistics("./table_m2a5000/")

In [ ]:
with open("./table_m2a1000/obj.pkl","rb") as file:
    obj = pickle.load(file)
    
obj.statistics()

In [ ]:
dnscase = oneDCaseReader("../Cases/nhep1d/dnsStandard/")
spv16 = dnscase.readData(0.0032,"SourcePV")
#spv05 = dnscase.readData(0.00055,"SourcePV")
x = np.linspace(-0.000512,0.000512,1000)
plt.plot(x,spv16,label = "spv16")
#plt.plot(x,spv05,label = "spv05")
plt.legend()

In [ ]:
c1dSpvDict = readOFScalarList("SourcePV_table","./table_c1d/")
c1dSpv = c1dSpvDict["data"]
c1dSpv = np.reshape(c1dSpv,(301,501)).transpose()

m3SpvDict  = readOFScalarList("SourcePV_table","./table_m3a5000/")
m3Spv  = m3SpvDict["data"]
m3Spv  = np.reshape(m3Spv, (301,501)).transpose()

plt.subplot(211)
plt.pcolor(c1dSpv)
plt.colorbar()
plt.subplot(212)
plt.pcolor(m3Spv)
plt.colorbar()

In [ ]:
m3SpvFinal = m3Spv[500,:]
c1dSpvFinal = c1dSpv[500,:]
plt.plot(m3SpvFinal, label = "method 3")
plt.plot(c1dSpvFinal, label = "chem1d")
plt.legend()

In [ ]:
m3SpvDict  = readOFScalarList("SourcePV_table","./table_m3a5000repair/")
m3Spv  = m3SpvDict["data"]
m3Spv  = np.reshape(m3Spv, (301,501)).transpose()
plt.plot(m3Spv[500,:])


m3Spv = m3Spv.transpose().reshape(-1)
m3SpvDict["data"] = m3Spv

writeOFScalarList(m3SpvDict,"./table_m3a5000repair/")